In [1]:

import json
import io
from datetime import datetime
from pytube import YouTube
import boto3
import sys
import logging
import os.path
import re
import xmltodict
from functools import reduce
import openai

S3 = boto3.client('s3', region_name="us-east-1")
dyndb = boto3.client('dynamodb', region_name="us-east-1")
ssm = boto3.client('ssm', region_name="us-east-1")

configs = ssm.get_parameter(Name="/civilton/youtube/config", WithDecryption=False)['Parameter']['Value'].split(",")

BUCKET_NAME = [config for config in configs if 'bucket_name' in config][0].split("=")[1]
URL_EXPIRATION = [config for config in configs if 'url_expiration' in config][0].split("=")[1]
CHAT_GPT_TOKEN = [config for config in configs if 'CHAT_GPT_TOKEN' in config][0].split("=")[1]
MAX_TOKENS = int([config for config in configs if 'max_tokens' in config][0].split("=")[1])
TEMPERATURE = float([config for config in configs if 'temperature' in config][0].split("=")[1])

openai.api_key = os.environ['CHAT_GPT_TOKEN']

In [5]:
def get_summary(text: str) -> str:
    max_total_tokens = 2049
    chunks = [text[i:i+max_total_tokens] for i in range(0, len(text), max_total_tokens)]
    max_summary_tokens = int(MAX_TOKENS / len(chunks))
    summaries = []
    for chunck in chunks:
        response = openai.Completion.create(
            engine="text-davinci-002",
            prompt=text,
            max_tokens=max_summary_tokens,
            temperature=TEMPERATURE,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0.6,
            stop=["\n", " Human:", " AI:"]
        )
        summaries.append(response.choices[0].text)
    summary = " ".join(summaries)
    return summary


In [6]:
text = "This is a test of the emergency broadcast system. This is only a test. If this had been an actual emergency, you would have been instructed to do something. This concludes this test of the emergency broadcast system."
get_summary(text)

''